## import

**packages**

In [1]:
from category_encoders import BinaryEncoder, TargetEncoder


# Initialize the target encoder
te = TargetEncoder(smoothing=5)


In [26]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns 
from sklearn.preprocessing import LabelEncoder,StandardScaler, OneHotEncoder, RobustScaler, MinMaxScaler      # Model Pre-Processing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, KFold, GridSearchCV,cross_val_score
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
from sklearn.pipeline import Pipeline
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from category_encoders import BinaryEncoder, TargetEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from catboost import CatBoostRegressor    # Regression Models
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from math import sqrt
from IPython.display import display, HTML # Display Preferences
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler, QuantileTransformer, PowerTransformer



**data**

In [3]:
# Import Raw Data
data_raw = pd.read_csv('data/Ames_Housing_Price_Data.csv')
data_raw.rename(columns={'Unnamed: 0': 'Unnamed 0'}, inplace=True)

# columns_to_drop = ['Unnamed: 0'] #'price_per_sqft',
# data_raw = data_raw.drop(columns=columns_to_drop)


**display prefs**

In [4]:
# display all rows
pd.set_option('display.max_rows', None)

# display all columns
pd.set_option('display.max_columns', None)

In [5]:
# Display Max Column and Rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Code Block Output: Enable 100px + scrolling
HTML("""
<style>
.output_scroll {
    box-sizing: border-box;
    display: block;
    -webkit-box-sizing: border-box;
    -moz-box-sizing: border-box;
    max-height: 100px;
    overflow: auto;
}
</style>
""")

**Notes**

In [6]:
# Create a DataFrame
model_preprocessing = pd.DataFrame({
    'Model': ['Regularized Linear Regression', 'Boosting Models (e.g., XGBoost, LightGBM)', 'Random Forest', 'Neural Networks', 'Support Vector Machines (SVM)', 'K-Nearest Neighbors (KNN)'],
    'Handling Categorical Variables': ['Required (One-hot Encoding, Label Encoding, etc.)', 'Required (One-hot Encoding, Label Encoding, etc.)', 'Required (One-hot Encoding, Label Encoding, etc.)', 'Required (One-hot Encoding, Label Encoding, etc.)', 'Required (One-hot Encoding, Label Encoding, etc.)', 'Required (One-hot Encoding, Label Encoding, etc.)'],
    'Handling Missing Values': ['Required', 'Required (some models like XGBoost can handle internally)', 'Required', 'Required', 'Required', 'Required'],
    'Feature Scaling': ['Required (especially for models like Lasso and Ridge)', 'Not Required (but can be beneficial in some cases, MinMaxScaler)', 'Not Required', 'Required (Neural networks are sensitive to feature scale)', 'Required (SVMs are sensitive to feature scale)', 'Required (KNN is sensitive to feature scale)'],
    'Dimensionality Reduction': ['Can be beneficial', 'Can be beneficial', 'Not always necessary', 'Can be beneficial', 'Can be beneficial', 'Can be beneficial']
})

# Display the DataFrame
model_preprocessing


,Model,Handling Categorical Variables,Handling Missing Values,Feature Scaling,Dimensionality Reduction
0,Regularized Linear Regression,"Required (One-hot Encoding, Label Encoding, etc.)",Required,Required (especially for models like Lasso and...,Can be beneficial
1,"Boosting Models (e.g., XGBoost, LightGBM)","Required (One-hot Encoding, Label Encoding, etc.)",Required (some models like XGBoost can handle ...,Not Required (but can be beneficial in some ca...,Can be beneficial
2,Random Forest,"Required (One-hot Encoding, Label Encoding, etc.)",Required,Not Required,Not always necessary
3,Neural Networks,"Required (One-hot Encoding, Label Encoding, etc.)",Required,Required (Neural networks are sensitive to fea...,Can be beneficial
4,Support Vector Machines (SVM),"Required (One-hot Encoding, Label Encoding, etc.)",Required,Required (SVMs are sensitive to feature scale),Can be beneficial
5,K-Nearest Neighbors (KNN),"Required (One-hot Encoding, Label Encoding, etc.)",Required,Required (KNN is sensitive to feature scale),Can be beneficial


## 0. Base Model performance

## 1. Drop

**drop unnamed columns**

In [7]:
columns_to_drop = ['Unnamed 0'] #'price_per_sqft',
data_raw = data_raw.drop(columns=columns_to_drop)


# 2: Encode

In [8]:
data_raw.select_dtypes(include='object').columns.tolist()

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

**encode after split**

In [9]:
# Assume that 'SalePrice' is your target column
target_encoding = data_raw.copy()
X = target_encoding.drop('SalePrice', axis=1)
y = target_encoding['SalePrice']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the target encoder
te = TargetEncoder(smoothing=5)

# Fit the encoder on the training data and transform both the training and test data
X_train_encoded = te.fit_transform(X_train, y_train)
X_test_encoded = te.transform(X_test)

# Initialize the CatBoostRegressor
model = CatBoostRegressor(verbose=0)

# Train the model
model.fit(X_train_encoded, y_train)

# Make predictions on the training and test sets
train_predictions = model.predict(X_train_encoded)
test_predictions = model.predict(X_test_encoded)

# Calculate and print the R2 score, RMSE, and MAE
print("Train R2: ", r2_score(y_train, train_predictions))
print("Test R2: ", r2_score(y_test, test_predictions))
print("Train RMSE: ", sqrt(mean_squared_error(y_train, train_predictions)))
print("Test RMSE: ", sqrt(mean_squared_error(y_test, test_predictions)))
print("Train MAE: ", mean_absolute_error(y_train, train_predictions))
print("Test MAE: ", mean_absolute_error(y_test, test_predictions))


/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


Train R2:  0.9939731835568638
Test R2:  0.9425736824173715
Train RMSE:  5751.640523457935
Test RMSE:  18795.445012139557
Train MAE:  4492.233545167931
Test MAE:  11681.378556700603


In [10]:
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from math import sqrt

# Assume that 'SalePrice' is your target column
target_encoding = data_raw.copy()
X = target_encoding.drop('SalePrice', axis=1)
y = target_encoding['SalePrice']

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)  # Temp is combined validation + test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # Split temp into validation and test

# Initialize the target encoder
te = TargetEncoder(smoothing=5)

# Fit the encoder on the training data and transform training, validation, and test data
X_train_encoded = te.fit_transform(X_train, y_train)
X_val_encoded = te.transform(X_val)
X_test_encoded = te.transform(X_test)

# Initialize the CatBoostRegressor
model = CatBoostRegressor(verbose=0)

# Train the model using training and validation sets
model.fit(X_train_encoded, y_train, eval_set=(X_val_encoded, y_val))

# Make predictions on the training, validation and test sets
train_predictions = model.predict(X_train_encoded)
val_predictions = model.predict(X_val_encoded)
test_predictions = model.predict(X_test_encoded)

# Calculate and print the R2 score, RMSE, and MAE
print("Train R2: ", r2_score(y_train, train_predictions))
print("Val R2: ", r2_score(y_val, val_predictions))
print("Test R2: ", r2_score(y_test, test_predictions))
print("Train RMSE: ", sqrt(mean_squared_error(y_train, train_predictions)))
print("Val RMSE: ", sqrt(mean_squared_error(y_val, val_predictions)))
print("Test RMSE: ", sqrt(mean_squared_error(y_test, test_predictions)))
print("Train MAE: ", mean_absolute_error(y_train, train_predictions))
print("Val MAE: ", mean_absolute_error(y_val, val_predictions))
print("Test MAE: ", mean_absolute_error(y_test, test_predictions))

print


/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


Train R2:  0.9957451903945113
Val R2:  0.9284089096365036
Test R2:  0.9454605793360498
Train RMSE:  4718.2785416957195
Val RMSE:  22904.094600137927
Test RMSE:  17485.07861131927
Train MAE:  3691.3500914557685
Val MAE:  12873.2097385998
Test MAE:  11306.61852578595


<function print>

In [11]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'depth': [3, 4, 5],
    'learning_rate': [0.01,0.05],
    'iterations': [500,1000]
}

# Initialize CatBoostRegressor
model = CatBoostRegressor(verbose=0)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

# Fit data to Grid Search
grid_search.fit(X_train_encoded, y_train)

# Get the best parameters
best_params = grid_search.best_params_

# Get the best score
best_score = grid_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best score: {best_score}")

# Fit the model with the best parameters found
model = CatBoostRegressor(**best_params, verbose=0)
model.fit(X_train_encoded, y_train, eval_set=(X_val_encoded, y_val))

# Continue with your predictions and evaluation

# Make predictions on the training, validation and test sets
train_predictions = model.predict(X_train_encoded)
val_predictions = model.predict(X_val_encoded)
test_predictions = model.predict(X_test_encoded)

# Calculate and print the R2 score, RMSE, and MAE
print("Train R2: ", r2_score(y_train, train_predictions))
print("Val R2: ", r2_score(y_val, val_predictions))
print("Test R2: ", r2_score(y_test, test_predictions))
print("Train RMSE: ", sqrt(mean_squared_error(y_train, train_predictions)))
print("Val RMSE: ", sqrt(mean_squared_error(y_val, val_predictions)))
print("Test RMSE: ", sqrt(mean_squared_error(y_test, test_predictions)))
print("Train MAE: ", mean_absolute_error(y_train, train_predictions))
print("Val MAE: ", mean_absolute_error(y_val, val_predictions))
print("Test MAE: ", mean_absolute_error(y_test, test_predictions))



/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


Best parameters: {'depth': 5, 'iterations': 1000, 'learning_rate': 0.05}
Best score: 0.915360709991854
Train R2:  0.9910002063867195
Val R2:  0.9314685015561626
Test R2:  0.9448840672379218
Train RMSE:  6862.143335107635
Val RMSE:  22409.323916657195
Test RMSE:  17577.24919498498
Train MAE:  5380.085472734284
Val MAE:  13101.01293157815
Test MAE:  11359.18460822544


**label encoding**

In [12]:
label_encoding = data_raw.copy()
dummified_encoding = data_raw.copy()
ordinal_encoding = data_raw.copy()
binary_encoding = data_raw.copy()
frequency_encoding = data_raw.copy()
target_encoding = data_raw.copy()
hashing_encoding = data_raw.copy()


In [13]:
# Ordinal Encoding
# Note: You need to specify the order of categories for each ordinal feature
# oe = OrdinalEncoder(categories=[['low', 'medium', 'high']])
# ordinal_encoding['Quality'] = oe.fit_transform(ordinal_encoding[['Quality']])

# Label Encoding
le = LabelEncoder()
for col in label_encoding.columns:
    if label_encoding[col].dtype == 'object':
        label_encoding[col] = le.fit_transform(label_encoding[col])

# Dummified
dummified_encoding = pd.get_dummies(dummified_encoding)

# Binary Encoding
be = BinaryEncoder()
binary_encoding = be.fit_transform(binary_encoding)

# Frequency Encoding
for col in frequency_encoding.columns:
    if frequency_encoding[col].dtype == 'object':
        freq = frequency_encoding[col].value_counts(normalize=True)
        frequency_encoding[col] = frequency_encoding[col].map(freq)

# Hashing Encoding
fh = FeatureHasher(n_features=10, input_type='string')
for col in hashing_encoding.columns:
    if hashing_encoding[col].dtype == 'object':
        hashed_features = fh.transform(hashing_encoding[[col]].astype(str).values)
        hashed_features = pd.DataFrame(hashed_features.toarray(), columns=[f"{col}_{i}" for i in range(10)])
        hashing_encoding = pd.concat([hashing_encoding, hashed_features], axis=1)
        hashing_encoding = hashing_encoding.drop(col, axis=1)

# Target Encoding with smoothing
te = TargetEncoder(smoothing=10)  # adjust the smoothing parameter as needed
for col in target_encoding.drop(columns='SalePrice').columns:
    if target_encoding[col].dtype == 'object':
        target_encoding[col] = te.fit_transform(target_encoding[col], target_encoding['SalePrice'])
        
# Backward Difference Encoding
backward_difference_encoding = data_raw.copy()
encoder = ce.BackwardDifferenceEncoder()
backward_difference_encoding = encoder.fit_transform(backward_difference_encoding)

# Polynomial Encoding
polynomial_encoding = data_raw.copy()
encoder = ce.PolynomialEncoder()
polynomial_encoding = encoder.fit_transform(polynomial_encoding)

# Helmert Encoding
helmert_encoding = data_raw.copy()
encoder = ce.HelmertEncoder()
helmert_encoding = encoder.fit_transform(helmert_encoding)

# Sum Encoding
sum_encoding = data_raw.copy()
encoder = ce.SumEncoder()
sum_encoding = encoder.fit_transform(sum_encoding)

# One-Hot Encoding with sklearn
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
onehot_encoding_sklearn = data_raw.copy()
# Select only the object columns
object_cols = onehot_encoding_sklearn.select_dtypes(include='object').columns
# Fit and transform the object columns
onehot_encoded_cols = ohe.fit_transform(onehot_encoding_sklearn[object_cols])
# The output of the transformation is a NumPy array, so we need to convert it back to a DataFrame
onehot_encoded_cols_df = pd.DataFrame(onehot_encoded_cols, columns=ohe.get_feature_names_out(object_cols))
# Drop the original object columns from the original DataFrame
onehot_encoding_sklearn = onehot_encoding_sklearn.drop(object_cols, axis=1)
# Concatenate the original DataFrame with the one-hot encoded columns
onehot_encoding_sklearn = pd.concat([onehot_encoding_sklearn, onehot_encoded_cols_df], axis=1)


     

/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/category_encoders/base_c

**run models**

In [14]:
# Define models
models = {
    "CatBoost": CatBoostRegressor(verbose=0),
    "LightGBM": LGBMRegressor(),
    "XGBoost": XGBRegressor()
}

# Define a function to evaluate models
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)
    return {
        "Train R2": r2_score(y_train, train_predictions),
        "Test R2": r2_score(y_test, test_predictions),
        "Train RMSE": sqrt(mean_squared_error(y_train, train_predictions)),
        "Test RMSE": sqrt(mean_squared_error(y_test, test_predictions)),
        "Train MAE": mean_absolute_error(y_train, train_predictions),
        "Test MAE": mean_absolute_error(y_test, test_predictions)
    }

# Define a function to prepare data and evaluate models
def prepare_and_evaluate(data):
    X = data.drop(columns='SalePrice')
    y = data['SalePrice']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return {name: evaluate_model(model, X_train, X_test, y_train, y_test) for name, model in models.items()}

# Prepare data and evaluate models
results = {
    "Label Encoding": prepare_and_evaluate(label_encoding),
    "Dummified Encoding": prepare_and_evaluate(dummified_encoding),
    "Binary Encoding": prepare_and_evaluate(binary_encoding),
    "Frequency Encoding": prepare_and_evaluate(frequency_encoding),
    "Target Encoding": prepare_and_evaluate(target_encoding),
    "Hashing Encoding": prepare_and_evaluate(hashing_encoding),
    "Backward Difference Encoding": prepare_and_evaluate(backward_difference_encoding),
    "Polynomial Encoding": prepare_and_evaluate(polynomial_encoding),
    "Helmert Encoding": prepare_and_evaluate(helmert_encoding),
    "Sum Encoding": prepare_and_evaluate(sum_encoding),
    "One-Hot Encoding": prepare_and_evaluate(onehot_encoding_sklearn)

}

# Flatten the results
flattened_results = {}
for encoding, models in results.items():
    for model, scores in models.items():
        for score, value in scores.items():
            flattened_results[(encoding, model, score)] = value

# Convert results to DataFrame
results_df = pd.DataFrame(flattened_results, index=[0])

# Transpose the DataFrame for better visualization
results_df = results_df.T.reset_index()

# Rename the columns
results_df.columns = ['Encoding', 'Model', 'Score', 'Value']

# Pivot the DataFrame to have models as columns and scores as rows for each encoding
results_df = results_df.pivot_table(index=['Score', 'Encoding'], columns='Model', values='Value')

print(results_df)


/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/catboost/core.py:1411: Futu

Model                                        CatBoost      LightGBM  \
Score      Encoding                                                   
Test MAE   Backward Difference Encoding  11913.793600  14151.743678   
           Binary Encoding               12202.661443  14145.097757   
           Dummified Encoding            12334.491225  13900.037137   
           Frequency Encoding            12542.878922  14164.724294   
           Hashing Encoding              12731.425122  14402.230365   
           Helmert Encoding              12012.119757  13968.093158   
           Label Encoding                12191.993787  13979.644723   
           One-Hot Encoding              12552.084293  14045.058854   
           Polynomial Encoding           12162.782098  13921.821498   
           Sum Encoding                  12561.407724  14062.712825   
           Target Encoding               11600.022031  13800.968747   
Test R2    Backward Difference Encoding      0.940118      0.914890   
      

In [15]:
results_df

Model                                        CatBoost      LightGBM  \
Score      Encoding                                                   
Test MAE   Backward Difference Encoding  11913.793600  14151.743678   
           Binary Encoding               12202.661443  14145.097757   
           Dummified Encoding            12334.491225  13900.037137   
           Frequency Encoding            12542.878922  14164.724294   
           Hashing Encoding              12731.425122  14402.230365   
           Helmert Encoding              12012.119757  13968.093158   
           Label Encoding                12191.993787  13979.644723   
           One-Hot Encoding              12552.084293  14045.058854   
           Polynomial Encoding           12162.782098  13921.821498   
           Sum Encoding                  12561.407724  14062.712825   
           Target Encoding               11600.022031  13800.968747   
Test R2    Backward Difference Encoding      0.940118      0.914890   
           Binary Encoding                   0.940078      0.913052   
           Dummified Encoding                0.939426      0.914435   
           Frequency Encoding                0.936843      0.915118   
           Hashing Encoding                  0.936348      0.910237   
           Helmert Encoding                  0.940888      0.913356   
           Label Encoding                    0.939315      0.914723   
           One-Hot Encoding                  0.939668      0.913281   
           Polynomial Encoding               0.938473      0.913846   
           Sum Encoding                      0.938726      0.913242   
           Target Encoding                   0.942905      0.918318   
Test RMSE  Backward Difference Encoding  19193.128323  22881.713036   
           Binary Encoding               19199.499482  23127.402392   
           Dummified Encoding            19303.671690  22942.716340   
           Frequency Encoding            19710.943360  22850.982528   
           Hashing Encoding              19788.115186  23498.768834   
           Helmert Encoding              19069.233683  23086.942787   
           Label Encoding                19321.416294  22904.036530   
           One-Hot Encoding              19265.144337  23096.918382   
           Polynomial Encoding           19454.945009  23021.631529   
           Sum Encoding                  19414.883523  23102.062395   
           Target Encoding               18741.208374  22416.167762   
Train MAE  Backward Difference Encoding   4569.630715   5709.008291   
           Binary Encoding                4534.509535   5669.683064   
           Dummified Encoding             5080.344279   5623.591922   
           Frequency Encoding             4611.170228   5543.028055   
           Hashing Encoding               4957.483488   5874.330223   
           Helmert Encoding               4963.153489   5542.385627   
           Label Encoding                 4607.636117   5679.790998   
           One-Hot Encoding               4997.338919   5660.019387   
           Polynomial Encoding            4498.581603   5295.487973   
           Sum Encoding                   4991.775615   5660.019387   
           Target Encoding                4482.125734   5508.865667   
Train R2   Backward Difference Encoding      0.993712      0.983512   
           Binary Encoding                   0.993777      0.983623   
           Dummified Encoding                0.992364      0.983765   
           Frequency Encoding                0.993520      0.984339   
           Hashing Encoding                  0.992701      0.982785   
           Helmert Encoding                  0.992743      0.984997   
           Label Encoding                    0.993513      0.984543   
           One-Hot Encoding                  0.992589      0.983736   
           Polynomial Encoding               0.993940      0.985052   
           Sum Encoding                      0.992576      0.983736   
           Target Encodi

## **3. Missing Values**

In [16]:
miss_val = data_raw.copy()

In [17]:
missing_values = miss_val.isnull().sum()
missing_values = missing_values[missing_values > 0]

missing_values_df = pd.DataFrame(missing_values, columns=['missing_values'])
missing_values_df['data_type'] = miss_val[missing_values.index].dtypes

missing_values_df.sort_values(by='data_type')


,missing_values,data_type
LotFrontage,462,float64
MasVnrArea,14,float64
GarageArea,1,float64
GarageCars,1,float64
BsmtFinSF1,1,float64
GarageYrBlt,129,float64
BsmtFinSF2,1,float64
BsmtUnfSF,1,float64
TotalBsmtSF,1,float64
BsmtFullBath,2,float64


In [18]:
# Lot Frontage: NaN Probably means 0 (0.0)
# MasVnrArea:   NaN Probably means 0 (0.0)
# GarageArea:   NaN Probably means 0 (0.0)
# BsmtFinSF1:   NaN Probably means 0 (0.0)
# GarageYrBlt:  NaN Probably means 0 (0000)
# BsmtUnfSF:    NaN Probably means 0 (0.0)
# TotalBsmtSF:  NaN Probably means 0 (0.0)
# BsmtFullBath: NaN Probably means 0 (0.0)
# BsmtHalfBath: NaN Probably means 0 (0.0)
# PoolQC:       NaN Probably means 'No Pool'. Will Replace with none
# GarageCond:   NaN Probably means 'No Garage'. Will Replace with none
# GarageQual:   NaN Probably means 'No Garage'. Will Replace with none
# GarageFinish: NaN Probably means 'No Garage'. Will Replace with none
# GarageType:   NaN Probably means 'No Garage'. Unusual because 2 less.
# Fence:        NaN Probably means 'No Fence'.  Will Replace with none
# BsmtFinType2: NaN Probably means 'No Bsmt'.   Will Replace with none
# BsmtFinType1: NaN Probably means 'No Bsmt'.   Will Replace with none
# BsmtExposure: NaN Probably means 'No Bsmt'. Will Replace with none
# BsmtCond:     NaN Probably means 'No Bsmt'. Will Replace with none
# BsmtQual:     NaN Probably means 'No Bsmt'. Will Replace with none
# Alley:        NaN Probably means 'No Alley'. Will Replace with none
# FireplaceQu:  NaN Probably means 'No Fireplace'. Will Replace with none
# MiscFeature:  NaN Probably means 'No MiscFeature'. Will Replace with none


In [19]:
for col in miss_val.columns:
    if miss_val[col].isnull().sum() > 0:     # are there are missing values?
        if miss_val[col].dtype == 'object':  # if the column is of object type fill 'Unk'
            miss_val[col].fillna('None', inplace=True)
        elif miss_val[col].dtype in ['int64', 'float64']:  # if the column is of numeric type
            miss_val[col].fillna(0.0, inplace=True)          # fill with 0


In [20]:
label_encoding = miss_val.copy()
dummified_encoding = miss_val.copy()
ordinal_encoding = miss_val.copy()
binary_encoding = miss_val.copy()
frequency_encoding = miss_val.copy()
hashing_encoding = miss_val.copy()
target_encoding = miss_val.copy()
backward_difference_encoding = miss_val.copy()
polynomial_encoding = miss_val.copy()
helmert_encoding = miss_val.copy()
sum_encoding = miss_val.copy()


In [21]:
# Ordinal Encoding
# Note: You need to specify the order of categories for each ordinal feature
# oe = OrdinalEncoder(categories=[['low', 'medium', 'high']])
# ordinal_encoding['Quality'] = oe.fit_transform(ordinal_encoding[['Quality']])

# Label Encoding
le = LabelEncoder()
for col in label_encoding.columns:
    if label_encoding[col].dtype == 'object':
        label_encoding[col] = le.fit_transform(label_encoding[col])

# Dummified
dummified_encoding = pd.get_dummies(dummified_encoding)

# Binary Encoding
be = BinaryEncoder()
binary_encoding = be.fit_transform(binary_encoding)

# Frequency Encoding
for col in frequency_encoding.columns:
    if frequency_encoding[col].dtype == 'object':
        freq = frequency_encoding[col].value_counts(normalize=True)
        frequency_encoding[col] = frequency_encoding[col].map(freq)

# Hashing Encoding
fh = FeatureHasher(n_features=10, input_type='string')
for col in hashing_encoding.columns:
    if hashing_encoding[col].dtype == 'object':
        hashed_features = fh.transform(hashing_encoding[[col]].astype(str).values)
        hashed_features = pd.DataFrame(hashed_features.toarray(), columns=[f"{col}_{i}" for i in range(10)])
        hashing_encoding = pd.concat([hashing_encoding, hashed_features], axis=1)
        hashing_encoding = hashing_encoding.drop(col, axis=1)

# Target Encoding with smoothing
te = TargetEncoder(smoothing=10)  # adjust the smoothing parameter as needed
for col in target_encoding.drop(columns='SalePrice').columns:
    if target_encoding[col].dtype == 'object':
        target_encoding[col] = te.fit_transform(target_encoding[col], target_encoding['SalePrice'])
        
# Backward Difference Encoding
encoder = ce.BackwardDifferenceEncoder()
backward_difference_encoding = encoder.fit_transform(backward_difference_encoding)

# Polynomial Encoding
encoder = ce.PolynomialEncoder()
polynomial_encoding = encoder.fit_transform(polynomial_encoding)

# Helmert Encoding
encoder = ce.HelmertEncoder()
helmert_encoding = encoder.fit_transform(helmert_encoding)

# Sum Encoding
encoder = ce.SumEncoder()
sum_encoding = encoder.fit_transform(sum_encoding)

# One-Hot Encoding with sklearn
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
onehot_encoding_sklearn = miss_val.copy()
# Select only the object columns
object_cols = onehot_encoding_sklearn.select_dtypes(include='object').columns
# Fit and transform the object columns
onehot_encoded_cols = ohe.fit_transform(onehot_encoding_sklearn[object_cols])
# The output of the transformation is a NumPy array, so we need to convert it back to a DataFrame
onehot_encoded_cols_df = pd.DataFrame(onehot_encoded_cols, columns=ohe.get_feature_names_out(object_cols))
# Drop the original object columns from the original DataFrame
onehot_encoding_sklearn = onehot_encoding_sklearn.drop(object_cols, axis=1)
# Concatenate the original DataFrame with the one-hot encoded columns
onehot_encoding_sklearn = pd.concat([onehot_encoding_sklearn, onehot_encoded_cols_df], axis=1)


     

/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/category_encoders/base_c

In [22]:
from sklearn.pipeline import make_pipeline


In [23]:
# Define models
models = {
    "CatBoost": CatBoostRegressor(verbose=0),
    "LightGBM": LGBMRegressor(),
    "XGBoost": XGBRegressor(),
    "Lasso": Lasso(),
    "ElasticNet": make_pipeline(ElasticNet()),
    "Ridge": Ridge(),
    "SVR": make_pipeline(SVR()),
    "RandomForest": RandomForestRegressor()
}

# Define a function to evaluate models
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)
    return {
        "Train R2": r2_score(y_train, train_predictions),
        "Test R2": r2_score(y_test, test_predictions),
        "Train RMSE": sqrt(mean_squared_error(y_train, train_predictions)),
        "Test RMSE": sqrt(mean_squared_error(y_test, test_predictions)),
        "Train MAE": mean_absolute_error(y_train, train_predictions),
        "Test MAE": mean_absolute_error(y_test, test_predictions)
    }

# Define a function to prepare data and evaluate models
def prepare_and_evaluate(data):
    X = data.drop(columns='SalePrice')
    y = data['SalePrice']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return {name: evaluate_model(model, X_train, X_test, y_train, y_test) for name, model in models.items()}

# Prepare data and evaluate models
results = {
    "Label Encoding": prepare_and_evaluate(label_encoding),
    "Dummified Encoding": prepare_and_evaluate(dummified_encoding),
    "Binary Encoding": prepare_and_evaluate(binary_encoding),
    "Frequency Encoding": prepare_and_evaluate(frequency_encoding),
    "Target Encoding": prepare_and_evaluate(target_encoding),
    "Hashing Encoding": prepare_and_evaluate(hashing_encoding),
    "Backward Difference Encoding": prepare_and_evaluate(backward_difference_encoding),
    "Polynomial Encoding": prepare_and_evaluate(polynomial_encoding),
    "Helmert Encoding": prepare_and_evaluate(helmert_encoding),
    "Sum Encoding": prepare_and_evaluate(sum_encoding),
    "One-Hot Encoding": prepare_and_evaluate(onehot_encoding_sklearn)

}

# Flatten the results
flattened_results = {}
for encoding, models in results.items():
    for model, scores in models.items():
        for score, value in scores.items():
            flattened_results[(encoding, model, score)] = value

# Convert results to DataFrame
results_df = pd.DataFrame(flattened_results, index=[0])

# Transpose the DataFrame for better visualization
results_df = results_df.T.reset_index()

# Rename the columns
results_df.columns = ['Encoding', 'Model', 'Score', 'Value']

# Pivot the DataFrame to have models as columns and scores as rows for each encoding
results_df = results_df.pivot_table(index=['Score', 'Encoding'], columns='Model', values='Value')

print(results_df)


/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.057e+11, tolerance: 1.133e+09
  model = cd_fast.enet_coordinate_descent(
/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.233e+11, tolerance: 1.133e+09

/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.285e+11, tolerance: 1.133e+09
  model = cd_fast.enet_coordinate_descent(
/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=9.63291e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_m

Model                                        CatBoost    ElasticNet  \
Score      Encoding                                                   
Test MAE   Backward Difference Encoding  12218.896118  16531.138609   
           Binary Encoding               12043.356304  17037.633495   
           Dummified Encoding            12498.476746  17017.683272   
           Frequency Encoding            12255.060993  18733.370265   
           Hashing Encoding              12460.727472  17209.649317   
           Helmert Encoding              11906.729468  14353.887984   
           Label Encoding                12480.852357  17546.398235   
           One-Hot Encoding              12498.476746  17017.683272   
           Polynomial Encoding           11951.099139  17017.674479   
           Sum Encoding                  12559.738165  16857.204493   
           Target Encoding               11884.479674  15870.434460   
Test R2    Backward Difference Encoding      0.939206      0.889882   
      

**Encoding Models Based on Column Type**

In [24]:
# List of variables
variables = ['MSSubClass', 'BldgType', 'HouseStyle', 'YearBuilt', 'YearRemodAdd', 'OverallQual', 'ExterQual', 
             'HeatingQC', 'BsmtFinType1', 'BsmtFinType2', 'KitchenQual', 'FireplaceQu', 'GarageQual', 
             'OverallCond', 'ExterCond', 'BsmtCond', 'Functional', 'GarageCond', 'Lot Frontage', 'Lot Area', 
             'LotShape', 'LandContour', 'LandSlope', 'LotConfig', 'Condition1', 'Condition2', 'RoofStyle', 
             'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'BsmtExposure', 'BsmtQual', 
             'Fireplaces', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', 
             '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 
             'HalfBath', 'Bedroom', 'Kitchen', 'TotRmsAbvGrd', 'BsmtFullBath', 'GarageType', 'GarageYrBlt', 
             'GarageFinish', 'GarageCars', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 
             'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscFeature', 'MiscVal', 'MSZoning', 'Street', 'Alley', 
             'Neighborhood', 'Utilities', 'Heating', 'CentralAir', 'Electrical', 'MoSold', 'YrSold', 'SaleType', 
             'SaleCondition']

# Define which variables are ordinal, nominal, and encoding
ordinal = ['OverallQual', 'ExterQual', 'HeatingQC', 'BsmtFinType1', 'BsmtFinType2', 'KitchenQual', 'FireplaceQu', 'GarageQual', 
           'OverallCond', 'ExterCond', 'BsmtCond', 'Functional', 'GarageCond', 'BsmtQual', 'GarageFinish', 'PoolQC']
nominal = ['MSZoning', 'Street', 'Alley', 'LandContour', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 
           'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 
           'Heating', 'CentralAir', 'Electrical', 'GarageType', 'PavedDrive', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
# encoding = [.....] # insert your list of variables that need encoding

# Create a dictionary
data = {'Variable': variables,
        'Ordinal': [1 if var in ordinal else 0 for var in variables],
        'Nominal': [1 if var in nominal else 0 for var in variables],
        'Encoding': [1 if var in encoding else 0 for var in variables]}

# Create DataFrame

df = pd.DataFrame(data)

print(df)


         Variable  Ordinal  Nominal  Encoding
0      MSSubClass        0        0         0
1        BldgType        0        1         0
2      HouseStyle        0        1         0
3       YearBuilt        0        0         0
4    YearRemodAdd        0        0         0
5     OverallQual        1        0         0
6       ExterQual        1        0         0
7       HeatingQC        1        0         0
8    BsmtFinType1        1        0         0
9    BsmtFinType2        1        0         0
10    KitchenQual        1        0         0
11    FireplaceQu        1        0         0
12     GarageQual        1        0         0
13    OverallCond        1        0         0
14      ExterCond        1        0         0
15       BsmtCond        1        0         0
16     Functional        1        0         0
17     GarageCond        1        0         0
18   Lot Frontage        0        0         0
19       Lot Area        0        0         0
20       LotShape        0        

## **scaling**

**handling infinites**

In [ ]:
# Define models
models = {
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "Ridge": Ridge(),
    "SVR": SVR(),
    "RandomForest": RandomForestRegressor(),
    "XGBoost": XGBRegressor(),
    "LightGBM": LGBMRegressor(),
    "CatBoost": CatBoostRegressor(verbose=0),
}

# Define scalers
scalers = {
    "StandardScaler": StandardScaler(),
    "RobustScaler": RobustScaler(),
    "MinMaxScaler": MinMaxScaler(),
    "MaxAbsScaler": MaxAbsScaler(),
    "QuantileTransformer": QuantileTransformer(),
    "PowerTransformer": PowerTransformer(),
}

# Define a function to evaluate models
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    predictions_train = model.predict(X_train)
    predictions_test = model.predict(X_test)
    return {
        "Train R2": r2_score(y_train, predictions_train),
        "Test R2": r2_score(y_test, predictions_test),
        "Train RMSE": sqrt(mean_squared_error(y_train, predictions_train)),
        "Test RMSE": sqrt(mean_squared_error(y_test, predictions_test)),
        "Train MAE": mean_absolute_error(y_train, predictions_train),
        "Test MAE": mean_absolute_error(y_test, predictions_test),
    }

def prepare_and_evaluate(data, scaler):
    # Prepare features and target
    X = data.drop('SalePrice', axis=1)
    y = data['SalePrice']
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Apply scaler and check for infinities
    try:
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        if np.isinf(X_train).any() or np.isinf(X_test).any():
            print('Infinities detected after scaling. Skipping this scaler...')
            return None
    except Exception as e:
        print(f"An error occurred: {e}. Skipping this scaler...")
        return None

    # Return the evaluation results
    return {name: evaluate_model(model, X_train, X_test, y_train, y_test) for name, model in models.items()}

# Define your encodings
encodings = {
    "Backward Difference Encoding": backward_difference_encoding,
    "Binary Encoding": binary_encoding,
    "Dummified Encoding": dummified_encoding,
    "Frequency Encoding": frequency_encoding,
    "Hashing Encoding": hashing_encoding,
    "Helmert Encoding": helmert_encoding,
    "Label Encoding": label_encoding,
    "One-Hot Encoding": onehot_encoding_sklearn,
    "Polynomial Encoding": polynomial_encoding,
    "Sum Encoding": sum_encoding,
    "Target Encoding": target_encoding
}

# Iterate over each encoding method and apply the prepare_and_evaluate function
results = {}
for encoding_name, encoded_data in encodings.items():
    print(f"Evaluating models with {encoding_name}...")
    for scaler_name, scaler in scalers.items():
        print(f"Applying {scaler_name} scaler...")
        result = prepare_and_evaluate(encoded_data, scaler)
        if result is not None:
            results[(encoding_name, scaler_name)] = result

# Convert the results to a DataFrame for better visualization
results_df = pd.DataFrame(results).T
print(results_df)



Evaluating models with Backward Difference Encoding...
Applying StandardScaler scaler...


/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.353e+11, tolerance: 1.133e+09
  model = cd_fast.enet_coordinate_descent(


Applying RobustScaler scaler...


/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.295e+11, tolerance: 1.133e+09
  model = cd_fast.enet_coordinate_descent(


Applying MinMaxScaler scaler...


...

/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.324e+10, tolerance: 1.133e+09
  model = cd_fast.enet_coordinate_descent(


Applying MaxAbsScaler scaler...


/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.640e+10, tolerance: 1.133e+09
  model = cd_fast.enet_coordinate_descent(


Applying QuantileTransformer scaler...


/Users/samuelminer/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.816e+10, tolerance: 1.133e+09
  model = cd_fast.enet_coordinate_descent(


In [ ]:
results_df.applymap(lambda x: x['Test R2'])

**catboost gridsearch**

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from numpy import sqrt
from numpy import linspace
from sklearn.model_selection import train_test_split

# Define your hyperparameters for CatBoost
hyperparameters = {
    "iterations": [100, 200, 300],
    "depth": [int(x) for x in linspace(4, 8, 3)],
    "learning_rate": linspace(0.01, 0.1, 3),
    "l2_leaf_reg": linspace(1, 9, 5),
}

# Custom scoring function
def custom_scorer(y_true, y_pred):
    rmse = sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    # Change these weights to your liking
    return r2 - (0.5 * rmse) - (0.5 * mae)

scorer = make_scorer(custom_scorer, greater_is_better=True)

# Prepare features and target
X = helmert_encoding.drop('SalePrice', axis=1)
y = helmert_encoding['SalePrice']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize CatBoostRegressor
model = CatBoostRegressor(verbose=0)

# Cross-validation with shuffle
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=hyperparameters,
    scoring=scorer,
    cv=cv,
    verbose=2,
    n_jobs=-1
)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Print best parameters
print("Best parameters found: ", grid_search.best_params_)

from sklearn.metrics import make_scorer

# Scoring function for MAE
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# Scoring function for RMSE
rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=False)

# Scoring function for R2
r2_scorer = make_scorer(r2_score, greater_is_better=True)

# Dictionary of scorers
scorers = {'r2': r2_scorer, 'mae': mae_scorer, 'rmse': rmse_scorer}

# More robust parameters for grid search
param_grid = {
    'iterations': [500, 1000],
    'depth': [3, 5],
    'learning_rate': np.linspace(.1,0.01),
}

# Grid search with CatBoost
model = CatBoostRegressor(loss_function='RMSE', verbose=0)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring=scorers, refit='r2', n_jobs=-1)

# Run grid search
grid_search.fit(X_train, y_train)

# Print the best parameters
print("Best Parameters: ", grid_search.best_params_)

# Fit the model with the best parameters
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test)

# Calculate and print the metrics
print("R2 Score: ", r2_score(y_test, y_pred))
print("Mean Absolute Error: ", mean_absolute_error(y_test, y_pred))
print("Root Mean Squared Error: ", sqrt(mean_squared_error(y_test, y_pred)))



# Archive

**mapping**

In [ ]:
mapped = data_raw.copy()

In [ ]:
columns = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 
           'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 
           'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 
           'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 
           'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 
           'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']

for column in columns:
    # Group the data by the column and calculate the mean of 'SalePrice' for each group
    average_price_per_group = mapped.groupby(column)['SalePrice'].mean()

    # Convert the Series to a DataFrame for better visualization
    average_price_per_group_df = average_price_per_group.reset_index()

    # Rename the columns
    average_price_per_group_df.columns = [column, 'AverageSalePrice']

    # Sort the DataFrame by 'AverageSalePrice'
    sorted_df = average_price_per_group_df.sort_values('AverageSalePrice')

    # Create a dictionary mapping the column to a number from 0 to 100
    column_map = {value: i * 100 / (len(sorted_df) - 1) for i, value in enumerate(sorted_df[column])}

    # Map 'column_map' to the column in 'mapped'
    mapped[column] = mapped[column].map(column_map)


In [ ]:
# Define target and features
X = mapped.drop(columns='SalePrice')
y = mapped['SalePrice']

# Create train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models
models = {
    "CatBoost": CatBoostRegressor(verbose=0),
    "LightGBM": LGBMRegressor(),
    "XGBoost": XGBRegressor(),
    "GradientBoosting": GradientBoostingRegressor()
}

# Define cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Function to evaluate models
def evaluate_model(model, X, y):
    cv_scores = cross_val_score(model, X, y, cv=cv, scoring='r2')
    print(f"R2: {cv_scores.mean()} (+/- {cv_scores.std() * 2})")
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print(f"MAE: {mean_absolute_error(y_test, predictions)}")
    print(f"RMSE: {sqrt(mean_squared_error(y_test, predictions))}")

# Evaluate models
for name, model in models.items():
    print(f"---- {name} ----")
    evaluate_model(model, X_train, y_train)
